In [80]:
pip install qeds fiona geopandas xgboost gensim folium pyLDAvis descartes statspy 

In [81]:
#Import
import pandas as pd
import numpy as np
import patsy
import statspy as sp
from sklearn import linear_model, ensemble, base, neural_network
#
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [82]:
#Load file - Demeaned by Location
from google.colab import drive
drive.mount('/content/drive')

df_path="/content/drive/My Drive/ML/KCYPS2010-m1e4e1_new.dta"
df=pd.read_stata(df_path)
df.describe()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,year,panel,gender,age,school_loc,ind_loc,school,grade,coeduw,f_edu,m_edu,health,exercise_time,ego_resilience1,ego_resilience2,ego_resilience3,ego_resilience4,ego_resilience5,ego_resilience6,ego_resilience7,ego_resilience8,ego_resilience9,ego_resilience10,ego_resilience11,ego_resilience12,ego_resilience13,ego_resilience14,satisfaction1,satisfaction2,satisfaction3,homework1_time1,homework1_time2,homework1_time3,homework1_time4,homework2_time1,homework2_time2,homework2_time3,homework2_time4,homework3_time1,...,temp_min_n,temp_min,temp_max_n,temp_max,rainfall_n,rainfall,wind_max_n,wind_max,wind_avg_n,wind_avg,m_study,f_study,female,exercise_time1,loc_yr,hwtime_schday,hwtime_nonschday,withfriend_time_sch,withfriend_time_nsch,satisfaction,aggressive,symptom_phy,depression,ego_resilience,social_cohesion,move,income,aggressive_p1,depression_p1,symptom_phy_p1,ego_resilience_p1,aggressive_p2,depression_p2,symptom_phy_p2,ego_resilience_p2,satisfaction_p3,aggressive_p3,depression_p3,symptom_phy_p3,ego_resilience_p3
count,25468.000000,25468.000000,25468.000000,25013.000000,25093.000000,19490.000000,25468.000000,25010.000000,25033.000000,13073.000000,24328.000000,24536.000000,25004.000000,25008.000000,13182.000000,13181.000000,13182.000000,13182.000000,13181.000000,13182.000000,13182.000000,13182.000000,13182.000000,13181.000000,13181.000000,13181.000000,13181.000000,13180.000000,23472.000000,23474.000000,23474.000000,24832.000000,24832.000000,24880.000000,24880.000000,24778.000000,24778.000000,24802.000000,24802.000000,24884.000000,...,24891.000000,24891.000000,24891.000000,24891.000000,24891.000000,24891.000000,24891.000000,24891.000000,24891.000000,24891.000000,25468.000000,25468.000000,25468.000000,25008.000000,25468.000000,24607.000000,24686.000000,24866.000000,24891.000000,23569.000000,16308.000000,16308.000000,16308.000000,13460.000000,24993.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000,25468.000000
mean,87913.105623,2012.853760,2.117952,1.467397,13.397601,2741.557414,2746.313570,1.862015,2.972636,2.451771,2.884865,2.698769,3.281715,2.913988,3.074040,3.036871,3.082385,3.043013,2.948866,3.035958,2.682749,2.949325,2.829237,3.037933,2.803657,2.780821,2.958197,3.173976,3.287960,2.791088,3.238434,0.427150,12.297922,0.427653,8.462379,0.483857,8.254702,0.473430,5.805338,0.847452,...,7.308627,3.960305,16.920568,13.723699,3.196835,1.948080,7.116713,7.314519,1.811941,1.880025,13.040051,13.441613,0.459047,1.913988,272.760803,2.211384,2.227036,0.841032,2.089274,0.001044,-0.000357,-0.000072,-0.001104,0.001772,0.000948,0.072444,4.713395,0.000290,-0.000158,0.000303,0.000600,0.000287,-0.000162,0.000298,0.000604,0.000726,0.000284,-0.000162,0.000298,0.000603
std,50006.514034,1.921218,0.757773,0.498946,2.308522,894.825103,895.389902,0.762092,1.736361,0.780727,1.789775,1.580844,0.593584,1.264765,0.650978,0.696299,0.771551,0.659711,0.868586,0.752759,0.917145,0.789322,0.789537,0.768452,0.811332,0.810767,0.776063,0.686779,0.682429,0.883672,0.724015,0.655464,14.657259,0.743685,13.559300,0.749488,13.700982,0.851854,12.110255,1.186044,...,3.089340,3.432631,2.439558,2.750279,4.701747,3.439408,2.055955,2.171004,0.962986,1.027465,3.835263,4.485485,0.498330,1.264866,147.699905,1.630425,2.155110,1.047577,2.007680,0.890658,0.890302,0.914353,0.940720,0.926650,0.890123,0.259226,2.632188,0.047453,0.058434,0.045313,0.070305,0.047391,0.058327,0.045203,0.070110,0.055151,0.047342,0.058376,0.045279,0.070275
min,14201.000000,2010.000000,1.000000,1.000000,9.000000,-9.000000,1001.000000,1.000000,1.000000,1.000000,-9.000000,-9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.983333,-5.644444,10.175000,5.533333,0.000000,

In [83]:
## (Outcome1) satisfaction1

#satisfaction, health, aggressive, depression

#Set dataset/cleaning
formula="""
satisfaction1 ~ pm10_bad + income + age + p_study + temp_avg + rainfall + wind_avg
"""

df["p_study"] = ((df["m_study"] >=12)|(df["f_study"] >=12)).astype(int)

satisfaction1, X = patsy.dmatrices(formula, df, return_type="dataframe")
# some categories are empty after dropping rows will Null, drop now
#X = X.loc[:, X.sum() > 0]
satisfaction1 = satisfaction1.iloc[:, 0]
treatment_variable = "pm10_bad"
treatment = X["pm10_bad"]
Xl = X.drop(["Intercept","pm10_bad"], axis=1)
loc_id = df.loc[X.index, "ind_loc"].astype("category")

import re
# remove [ ] from names for compatibility with xgboost
Xl = Xl.rename(columns=lambda x: re.sub('\[|\]','_',x))

In [105]:
################################################################################

##Step 1: Best ML 
# Def of SE
def get_treatment_se(fit, cluster_id, rows=None):
    if cluster_id is not None:
        if rows is None:
            rows = [True] * len(cluster_id)
        vcov = sm.stats.sandwich_covariance.cov_cluster(fit, cluster_id.loc[rows])
        return np.sqrt(np.diag(vcov))

    return fit.HC0_se

# Def of Generic ML model
def generic_ml_model(x, y, treatment, model, n_split=100, n_group=5, cluster_id=None):
    nobs = x.shape[0]

    blp = np.zeros((n_split, 2))
    blp_se = blp.copy()
    blp_ci_l = blp.copy()
    blp_ci_u = blp.copy()
    blp_pvalue= np.zeros((n_split, 4))

    gate = np.zeros((n_split, n_group))
    affected = np.zeros((n_split, 3))    
    gate_se = affected.copy()
    affected_ci_l = affected.copy()
    affected_ci_u = affected.copy()
    affected_pvalue = np.zeros((n_split, 6))

    blp_new = np.zeros((1, 2))
    affected_new = np.zeros((1, 3))

    baseline = np.zeros((nobs, n_split))
    cate = baseline.copy()
    lamb = np.zeros((n_split, 2))

    for i in range(n_split):
        main = np.random.rand(nobs) > 0.5
        rows1 = ~main & (treatment == 1)
        rows0 = ~main & (treatment == 0)

        mod1 = base.clone(model).fit(x.loc[rows1, :], (y.loc[rows1]))
        mod0 = base.clone(model).fit(x.loc[rows0, :], (y.loc[rows0]))

        B = mod0.predict(x)
        S = mod1.predict(x) - B

        if B.var()==0:
          B = B + np.random.normal(0, 0.1, nobs)
        if S.var()==0:
          S = S + np.random.normal(0, 0.1, nobs)

        baseline[:, i] = B
        cate[:, i] = S
        ES = S.mean()

        #Creating weights
        #p = treatment.mean()
        p=0.5
        num=main[main==1].shape[0]

        weight=np.repeat((1/(p*(1-p))), num)

################################################################################

        ## BLP
        # assume P(treat|x) = P(treat) = mean(treat)
        p = treatment.mean()
        reg_df = pd.DataFrame(dict(
            y=y, B=B, treatment=treatment, S=S, main=main, excess_S=S-ES
        ))
        reg =smf.wls("y ~ B + S + I(treatment-p) + I((treatment-p)*(S-ES))", data=reg_df.loc[main, :], weights = weight)
        #reg =smf.ols("y ~ B + I(treatment-p) + I((treatment-p)*(S-ES))", data=reg_df.loc[main, :]);
        reg_fit = reg.fit(cov_type='cluster', cov_kwds={'groups': cluster_id[main] })

        blp_se[i, :] = get_treatment_se(reg_fit, cluster_id, main)[3:]
        
        #beta1 BLP
        beta1_blp = reg_fit.params.iloc[3]
        beta1_blp_ci_l = reg_fit.conf_int(alpha=0.05, cols=None).iloc[:,0][3]
        beta1_blp_ci_u= reg_fit.conf_int(alpha=0.05, cols=None).iloc[:,1][3]

        #beta2 BLP
        beta2_blp = reg_fit.params.iloc[4]
        beta2_blp_ci_l = reg_fit.conf_int(alpha=0.05, cols=None).iloc[:,0][4]
        beta2_blp_ci_u= reg_fit.conf_int(alpha=0.05, cols=None).iloc[:,1][4]

        #beta1, beta2
        blp[i,:]=beta1_blp, beta2_blp

        #BLP pvalue
        blp1_pvalue=reg_fit.pvalues[2]
        blp2_pvalue=reg_fit.pvalues[3]

        blp1_1=(beta1_blp<0)*1*(blp1_pvalue/2) + (beta1_blp>0)*1*(1-blp1_pvalue/2)
        blp1_2=(beta1_blp<0)*1*(1-blp1_pvalue/2) + (beta1_blp>0)*1*(blp1_pvalue/2)
        #blp1=min(1, 4*min(blp1_1, blp1_2))

        blp2_1=(beta2_blp<0)*1*(blp2_pvalue/2) + (beta2_blp>0)*1*(1-blp2_pvalue/2)
        blp2_2=(beta2_blp<0)*1*(1-blp2_pvalue/2) + (beta2_blp>0)*1*(blp2_pvalue/2)
        #blp2=min(1, 4*min(blp2_1, blp2_2))

        blp_pvalue[i,:] = blp1_1, blp1_2, blp2_1, blp2_2

        blp_ci_l[i,:]=beta1_blp_ci_l, beta2_blp_ci_l
        blp_ci_u[i,:]=beta1_blp_ci_u, beta2_blp_ci_u

        lamb[i, 0] = reg_fit.params.iloc[-1]**2 * S.var()

################################################################################

        ## GATEs
        cutoffs = np.quantile(S, np.linspace(0,1, n_group + 1))
        cutoffs[-1] += 1
        for k in range(n_group):
            reg_df[f"G{k}"] = (cutoffs[k] <= S) & (S < cutoffs[k+1])

        g_form = "y ~ -1 + B + S + " + " + ".join([f"I((treatment-p)*G{k})" for k in range(n_group)])
        #g_form = "y ~ B + S + " + " + ".join([f"I((treatment-p)*G{k})" for k in range(n_group)])
        g_reg = smf.wls(g_form, data=reg_df.loc[main, :], weights = weight)
        #g_reg = smf.ols(g_form, data=reg_df.loc[main, :])
        g_fit = g_reg.fit(cov_type='cluster', cov_kwds={'groups': cluster_id[main]})
        gate[i, :] = g_fit.params.values[2:] #g_fit.params.filter(regex="G").values

        lamb[i, 1] = (gate[i,:]**2).sum()/n_group

################################################################################

        ## Most/Least affected
        S2 = S + np.random.normal(0, 0.00001, len(S))
        cutoffs=np.quantile(S2, np.linspace(0,1, n_group + 1))
        cutoffs[0]=cutoffs[0] - 0.001
        cutoffs[5]=cutoffs[5] + 0.001 

        for k in range(n_group):
          reg_df[f"G{k}"] = (cutoffs[k] <= S) & (S < cutoffs[k+1])

        g_form = "y ~ -1 + B + S + " + " + ".join([f"I((treatment-p)*G{k})" for k in range(n_group)])
        #g_form = "y ~ -1 + I((treatment-p)*G4) + I((treatment-p)*G0)" (right)
        #g_form = "y ~ " + " + ".join([f"I((treatment-p)*G{k})" for k in range(n_group)])
        g_reg = smf.wls(g_form, data=reg_df.loc[main, :], weights = weight)
        g_fit = g_reg.fit(cov_type='cluster', cov_kwds={'groups': cluster_id[main]})

        #Most affected 
        most_coef=g_fit.params[6]        
        most_pvalue=g_fit.pvalues[6]
        most_ci_l=g_fit.conf_int(alpha=0.05, cols=None).iloc[6,0]
        most_ci_u=g_fit.conf_int(alpha=0.05, cols=None).iloc[6,1]


        #Least affected 
        least_coef=g_fit.params[2]
        least_pvalue=g_fit.pvalues[2]
        least_ci_l=g_fit.conf_int(alpha=0.05, cols=None).iloc[2,0]
        least_ci_u=g_fit.conf_int(alpha=0.05, cols=None).iloc[2,1]

        #Difference
        test=g_fit.t_test('I((treatment - p) * G4) - I((treatment - p) * G0) = 0')
        dir(test)
        #diff_coef= test.effect
        diff_coef=most_coef-least_coef
        diff_pvalue=test.pvalue
        diff_ci_l=test.conf_int(alpha=0.05)[0][0]
        diff_ci_u=test.conf_int(alpha=0.05)[0][1]

        #pvalue
        most_p1=(most_coef<0)*1*(most_pvalue/2)+(most_coef>0)*1*(1-most_pvalue/2)
        most_p2=(most_coef<0)*1*(1-most_pvalue/2)+(most_coef>0)*1*(most_pvalue/2)
        least_p1=(least_coef<0)*1*(least_pvalue/2)+(least_coef>0)*1*(1-least_pvalue/2)
        least_p2=(least_coef<0)*1*(1-least_pvalue/2)+(least_coef>0)*1*(least_pvalue/2)
        diff_p1=(diff_coef<0)*1*(diff_pvalue/2)+(diff_coef>0)*1*(1-diff_pvalue/2)
        diff_p2=(diff_coef<0)*1*(1-diff_pvalue/2)+(diff_coef>0)*1*(diff_pvalue/2)

        affected[i,:]=most_coef,least_coef,diff_coef
        affected_ci_l[i,:]=most_ci_l,least_ci_l,diff_ci_l
        affected_ci_u[i,:]=most_ci_u,least_ci_u,diff_ci_u
        affected_pvalue[i,:]=most_p1, most_p2, least_p1, least_p2, diff_p1, diff_p2

################################################################################

    blp_pvalue= np.nanmedian(blp_pvalue, axis=0)
    affected_pvalue= np.nanmedian(affected_pvalue, axis=0)

    blp_new[0][0] = min(1, 4*min(blp_pvalue[0], blp_pvalue[1]))
    blp_new[0][1] = min(1, 4*min(blp_pvalue[2], blp_pvalue[3]))
    blp_pvalue = blp_new

    affected_new[0][0] = min(1, 4*min(affected_pvalue[0], affected_pvalue[1]))
    affected_new[0][1] = min(1, 4*min(affected_pvalue[2], affected_pvalue[3]))
    affected_new[0][2] = min(1, 4*min(affected_pvalue[4], affected_pvalue[5]))
    affected_pvalue = affected_new


    out = dict(
        blp = blp,
        blp_ci_l=blp_ci_l, 
        blp_ci_u=blp_ci_u,
        blp_pvalue=blp_pvalue, 
        gate=gate,
        gate_se=gate_se, 
        blp_se=blp_se,
        affected = affected,
        affected_ci_l =affected_ci_l, 
        affected_ci_u= affected_ci_u, 
        affected_pvalue = affected_pvalue,
        Lambda=lamb, 
        baseline=baseline, 
        cate=cate,
        name=type(model).__name__
    )
    return out

In [106]:
# Def of Generic ML summary
def generic_ml_summary(generic_ml_output):
    out = {
        x: np.nanmedian(generic_ml_output[x], axis=0)
        for x in ["blp", "blp_ci_l", "blp_ci_u", "Lambda", "gate",
                  "affected", "affected_ci_l", "affected_ci_u", "gate_se", "blp_se"]
    }

    out.update({"blp_pvalue": generic_ml_output["blp_pvalue"]})
    out.update({"affected_pvalue": generic_ml_output["affected_pvalue"]})

    out["name"] = generic_ml_output["name"]
    return out

In [107]:
#Step 1: Tune ML hyperparameters
import xgboost as xgb
models = [
    linear_model.ElasticNetCV(cv=5, n_alphas=25, max_iter=1000, tol=1e-3, n_jobs=-1, l1_ratio=0.9),
    ensemble.RandomForestRegressor(n_estimators=100, min_samples_leaf=20),
    xgb.XGBRegressor(n_estimators=100, max_depth=3, reg_lambda=2.0, reg_alpha=0.0, objective="reg:squarederror"),
    neural_network.MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, activation="relu", solver="adam", tol=1e-3, early_stopping=True, alpha=0.0001, learning_rate="constant", shuffle=True)
]

#Set parameters
x=Xl
treatment=treatment
n_split= 100
cluster_id=loc_id

#Tune parameters
y=satisfaction1
n_group=5

In [108]:
##Step 1: Best BLP / GATES - choose best ML method
##Best BLP 
model_a=models[0]
model_b=models[1]
model_c=models[2]
model_d=models[3]

result1=generic_ml_model(x, y, treatment, model_a, n_split, n_group, cluster_id=loc_id)
ElasticNet_BestBLP=generic_ml_summary(result1)["Lambda"][0]

result2=generic_ml_model(x, y, treatment, model_b, n_split, n_group, cluster_id=loc_id)
RandomForest_BestBLP=generic_ml_summary(result2)["Lambda"][0]

result3=generic_ml_model(x, y, treatment, model_c, n_split, n_group, cluster_id=loc_id)
Booster_BestBLP=generic_ml_summary(result3)["Lambda"][0]

result4=generic_ml_model(x, y, treatment, model_d, n_split, n_group, cluster_id=loc_id)
neural_network_BestBLP=generic_ml_summary(result4)["Lambda"][0]

print (ElasticNet_BestBLP, RandomForest_BestBLP, Booster_BestBLP, neural_network_BestBLP)
#Boosting/Random Forest Best

##p value##
blp_pvalue1=result1["blp_pvalue"]
blp_pvalue2=result2["blp_pvalue"]
blp_pvalue3=result3["blp_pvalue"]
blp_pvalue4=result4["blp_pvalue"]

affected_pvalue1=result1["affected_pvalue"]
affected_pvalue2=result2["affected_pvalue"]
affected_pvalue3=result3["affected_pvalue"]
affected_pvalue4=result4["affected_pvalue"]


##Best GATE
ElasticNet_BestGATE=generic_ml_summary(result1)["Lambda"][1]
RandomForest_BestGATE=generic_ml_summary(result2)["Lambda"][1]
Booster_BestGATE=generic_ml_summary(result3)["Lambda"][1]
neural_network_BestGATE=generic_ml_summary(result4)["Lambda"][1]

#Stage 1: Create Table

best = {'Elastic Net':  [ElasticNet_BestBLP, ElasticNet_BestGATE],
        'Boosting': [Booster_BestBLP, Booster_BestGATE],
        'Neural Network': [neural_network_BestBLP, neural_network_BestGATE],
        'Random Forest': [RandomForest_BestBLP, RandomForest_BestGATE]
        }

Best_df = pd.DataFrame (best, columns = ['Elastic Net','Boosting','Neural Network','Random Forest'], index=['Best BLP','Best GATES'])
print(Best_df)

Best_df.to_csv("/content/drive/My Drive/ML/new2/Stage1_Comparison_satisfaction1.csv", index=True, header=True)


0.0011502344033280255 0.006678230275293941 0.009893323049381574 0.00451446257378805
            Elastic Net  Boosting  Neural Network  Random Forest
Best BLP       0.001150  0.009893        0.004514       0.006678
Best GATES     0.011488  0.017132        0.011709       0.012700


In [109]:
##Choose the best 2 ML methods

#Empty list
model_blp_final=[0,0]
model_gate_final=[0,0]

#BLP
blp_ml=[ElasticNet_BestBLP, RandomForest_BestBLP, Booster_BestBLP, neural_network_BestBLP]

# Choose Best BLP
if max(blp_ml)==ElasticNet_BestBLP:
  model_blp_final[0]=models[0]
elif max(blp_ml)== RandomForest_BestBLP:
  model_blp_final[0]=models[1]
elif max(blp_ml)== Booster_BestBLP:
  model_blp_final[0]=models[2]
else:
  model_blp_final[0]=models[3]

import heapq
if heapq.nlargest(2, blp_ml)[1]==ElasticNet_BestBLP:
  model_blp_final[1]=models[0]
elif heapq.nlargest(2, blp_ml)[1]== RandomForest_BestBLP:
  model_blp_final[1]=models[1]
elif heapq.nlargest(2, blp_ml)[1]== Booster_BestBLP:
  model_blp_final[1]=models[2]
else:
  model_blp_final[1]=models[3]

#GATE
gate_ml=[ElasticNet_BestGATE, RandomForest_BestGATE, Booster_BestGATE, neural_network_BestGATE]

#Choose Best GATE
if max(gate_ml)==ElasticNet_BestGATE:
  model_gate_final[0]=models[0]
elif max(gate_ml)== RandomForest_BestGATE:
  model_gate_final[0]=models[1]
elif max(gate_ml)== Booster_BestGATE:
  model_gate_final[0]=models[2]
else:
  model_gate_final[0]=models[3]

import heapq
if heapq.nlargest(2, gate_ml)[1]==ElasticNet_BestGATE:
  model_gate_final[1]=models[0]
elif heapq.nlargest(2, gate_ml)[1]== RandomForest_BestGATE:
  model_gate_final[1]=models[1]
elif heapq.nlargest(2, gate_ml)[1]== Booster_BestGATE:
  model_gate_final[1]=models[2]
else:
  model_gate_final[1]=models[3]

#Change orders 
#model_blp_final[0], model_blp_final[1] = model_blp_final[1], model_blp_final[0]
#model_gate_final[0], model_gate_final[1] = model_gate_final[1], model_gate_final[0]

print(model_blp_final,model_gate_final)

assert model_blp_final==model_gate_final

[XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0.0, reg_lambda=2.0, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1), RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=20,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)] [XGBReg

In [110]:
## Stage 2 : BLP & GATE of CATE
#Def of reporting results 

kw = dict(x=x, treatment= treatment, n_split=n_split, n_group=n_group, cluster_id=cluster_id)
@ignore_warnings(category=ConvergenceWarning)


def evaluate_models(models, y, **other_kw):
    all_kw = kw.copy()
    all_kw["y"] = y
    all_kw.update(other_kw)
    return list(map(lambda x: generic_ml_model(model=x, **all_kw), models))


def generate_report(results):
    summaries = list(map(generic_ml_summary, results))
    df_plot = pd.DataFrame({
        mod["name"]: np.median(mod["cate"], axis=1)
        for mod in results
    })

    print("\n\nBest linear projection of CATE")
    df_cate = pd.concat({
        s["name"]: pd.DataFrame(dict(blp=s["blp"], se=s["blp_se"], lower=s["blp_ci_l"], upper=s["blp_ci_u"]))
        for s in summaries
    }).T.stack()
    display(df_cate)
    df_cate.to_csv("/content/drive/My Drive/ML/new2/Stage2_BLP_satisfaction1.csv", index=True, header=True)

    print("\n\nGroup average treatment effects:")
    df_groups = pd.concat({
        s["name"]: pd.DataFrame(dict(gate=s["affected"], se=s["gate_se"], lower=s["affected_ci_l"], upper=s["affected_ci_u"]))
        for s in summaries
    }).T.stack()
    display(df_groups)
    df_groups.to_csv("/content/drive/My Drive/ML/new2/Stage2_gate_satisfaction1.csv", index=True, header=True)

#Only 2 Best ML method

results = evaluate_models(models=model_blp_final, y=y)
generate_report(results)



Best linear projection of CATE


XGBRegressor  RandomForestRegressor
blp   0     -0.080047              -0.078379
      1      0.660449               0.444939
se    0      0.017106               0.016566
      1      0.099868               0.084099
lower 0     -0.115114              -0.111040
      1      0.453741               0.282355
upper 0     -0.046560              -0.045539
      1      0.849959               0.617569



Group average treatment effects:


XGBRegressor  RandomForestRegressor
gate  0      0.057881               0.018675
      1     -0.227709              -0.194208
      2      0.288707               0.204645
se    0      0.000000               0.000000
      1      0.000000               0.000000
      2      0.000000               0.000000
lower 0     -0.008282              -0.046088
      1     -0.294270              -0.261629
      2      0.193615               0.115500
upper 0      0.124010               0.081680
      1     -0.165116              -0.126828
      2      0.379203               0.301151

In [111]:
# Stage 2: Create Table of BLP -BLP
#BLP
blp_path="/content/drive/My Drive/ML/new2/Stage2_BLP_satisfaction1.csv"
blp=pd.read_csv(blp_path)

#Rename ML methods
if blp.columns[2]=='ElasticNetCV':
  blp.rename(columns={'ElasticNetCV':'Elastic Net'}, inplace=True)
elif blp.columns[2]=='MLPRegressor':
  blp.rename(columns={'MLPRegressor':'Neural Network'}, inplace=True)
elif blp.columns[2]=='RandomForestRegressor':
  blp.rename(columns={'RandomForestRegressor':'Random Forest'}, inplace=True)
else:
  blp.rename(columns={'XGBRegressor':'Booster'}, inplace=True)

if blp.columns[3]=='ElasticNetCV':
  blp.rename(columns={'ElasticNetCV':'Elastic Net'}, inplace=True)
elif blp.columns[3]=='MLPRegressor':
  blp.rename(columns={'MLPRegressor':'Neural Network'}, inplace=True)
elif blp.columns[3]=='RandomForestRegressor':
  blp.rename(columns={'RandomForestRegressor':'Random Forest'}, inplace=True)
else:
  blp.rename(columns={'XGBRegressor':'Booster'}, inplace=True)

#Create Dataframe
a = np.arange(12).reshape(3,4)
a_df=pd.DataFrame(a)

#1st row: BLP
a_df.iloc[0,0]=round(blp.iloc[0,2],4)
a_df.iloc[0,1]=round(blp.iloc[1,2],4)
a_df.iloc[0,2]=round(blp.iloc[0,3],4)
a_df.iloc[0,3]=round(blp.iloc[1,3],4)

#2nd row: CI

a_df.iloc[1,0]=f"({round(blp.iloc[4,2],4)},{round(blp.iloc[6,2],4)})"
a_df.iloc[1,1]=f"({round(blp.iloc[5,2],4)},{round(blp.iloc[7,2],4)})"
a_df.iloc[1,2]=f"({round(blp.iloc[4,3],4)},{round(blp.iloc[6,3],4)})"
a_df.iloc[1,3]=f"({round(blp.iloc[5,3],4)},{round(blp.iloc[7,3],4)})"

#3rd row: SE 
a_df.iloc[2,0]=f"[{round(blp.iloc[2,2],4)}]"
a_df.iloc[2,1]=f"[{round(blp.iloc[3,2],4)}]"
a_df.iloc[2,2]=f"[{round(blp.iloc[2,3],4)}]"
a_df.iloc[2,3]=f"[{round(blp.iloc[3,3],4)}]"

a_df.rename(columns={0:f"{blp.columns[2]} ATE", 1:f"{blp.columns[2]} HET", 2:f"{blp.columns[3]} ATE",3:f"{blp.columns[3]} HET" },inplace=True)

print(a_df)
a_df.to_csv("/content/drive/My Drive/ML/new2/Stage2_BLP_satisfaction1.csv", index=False, header=True)

#blp_data = pd.DataFrame(columns=['ATE(beta1)', 'HET(beta2)', 'ATE(beta1)', 'HET(beta2)'], index=False)
#blp_data
#GATES

         Booster ATE    Booster HET Random Forest ATE Random Forest HET
0              -0.08         0.6604           -0.0784            0.4449
1  (-0.1151,-0.0466)  (0.4537,0.85)  (-0.111,-0.0455)   (0.2824,0.6176)
2           [0.0171]       [0.0999]          [0.0166]          [0.0841]


In [112]:
#BLP - pvalue substitution
if a_df.columns[0]== 'Elastic Net ATE':
  a_df.iloc[2,0]='[{:.4f}]'.format(blp_pvalue1[0][0])
  a_df.iloc[2,1]='[{:.4f}]'.format(blp_pvalue1[0][1])
elif a_df.columns[0]== 'Random Forest ATE':
  a_df.iloc[2,0]='[{:.4f}]'.format(blp_pvalue2[0][0])
  a_df.iloc[2,1]='[{:.4f}]'.format(blp_pvalue2[0][1])
elif a_df.columns[0]== 'Booster ATE':
  a_df.iloc[2,0]='[{:.4f}]'.format(blp_pvalue3[0][0])
  a_df.iloc[2,1]='[{:.4f}]'.format(blp_pvalue3[0][1])
else: 
  a_df.iloc[2,0]='[{:.4f}]'.format(blp_pvalue4[0][0])
  a_df.iloc[2,1]='[{:.4f}]'.format(blp_pvalue4[0][1])


if a_df.columns[2]== 'Elastic Net HET':
  a_df.iloc[2,2]='[{:.4f}]'.format(blp_pvalue1[0][0])
  a_df.iloc[2,3]='[{:.4f}]'.format(blp_pvalue1[0][1])
elif a_df.columns[2]== 'Random Forest HET':
  a_df.iloc[2,2]='[{:.4f}]'.format(blp_pvalue2[0][0])
  a_df.iloc[2,3]='[{:.4f}]'.format(blp_pvalue2[0][1])
elif a_df.columns[2]== 'Booster HET':
  a_df.iloc[2,2]='[{:.4f}]'.format(blp_pvalue3[0][0])
  a_df.iloc[2,3]='[{:.4f}]'.format(blp_pvalue3[0][1])
else: 
  a_df.iloc[2,2]='[{:.4f}]'.format(blp_pvalue4[0][0])
  a_df.iloc[2,3]='[{:.4f}]'.format(blp_pvalue4[0][1])


print(a_df)

a_df.to_csv("/content/drive/My Drive/ML/new2/Stage2_BLP_satisfaction1.csv", index=False, header=True)

         Booster ATE    Booster HET Random Forest ATE Random Forest HET
0              -0.08         0.6604           -0.0784            0.4449
1  (-0.1151,-0.0466)  (0.4537,0.85)  (-0.111,-0.0455)   (0.2824,0.6176)
2           [0.0000]       [0.0000]          [0.1138]          [0.0001]


In [113]:
# Stage 2: Create Table of GATE
#BLP
gate_path="/content/drive/My Drive/ML/new2/Stage2_gate_satisfaction1.csv"
gate=pd.read_csv(gate_path)

#Rename ML methods
if gate.columns[2]=='ElasticNetCV':
  gate.rename(columns={'ElasticNetCV':'Elastic Net'}, inplace=True)
elif gate.columns[2]=='MLPRegressor':
  gate.rename(columns={'MLPRegressor':'Neural Network'}, inplace=True)
elif gate.columns[2]=='RandomForestRegressor':
  gate.rename(columns={'RandomForestRegressor':'Random Forest'}, inplace=True)
else:
  gate.rename(columns={'XGBRegressor':'Booster'}, inplace=True)

if gate.columns[3]=='ElasticNetCV':
  gate.rename(columns={'ElasticNetCV':'Elastic Net'}, inplace=True)
elif gate.columns[3]=='MLPRegressor':
  gate.rename(columns={'MLPRegressor':'Neural Network'}, inplace=True)
elif gate.columns[3]=='RandomForestRegressor':
  gate.rename(columns={'RandomForestRegressor':'Random Forest'}, inplace=True)
else:
  gate.rename(columns={'XGBRegressor':'Booster'}, inplace=True)

#Create Dataframe
b = np.arange(18).reshape(3,6)
b_df=pd.DataFrame(b)

#1st row: gate
b_df.iloc[0,0]=round(gate.iloc[0,2],4)
b_df.iloc[0,1]=round(gate.iloc[1,2],4)
b_df.iloc[0,2]=round(gate.iloc[2,2],4)
b_df.iloc[0,3]=round(gate.iloc[0,3],4)
b_df.iloc[0,4]=round(gate.iloc[1,3],4)
b_df.iloc[0,5]=round(gate.iloc[2,3],4)

#2nd row: CI
b_df.iloc[1,0]=f"({round(gate.iloc[6,2],4)},{round(gate.iloc[9,2],4)})"
b_df.iloc[1,1]=f"({round(gate.iloc[7,2],4)},{round(gate.iloc[10,2],4)})"
b_df.iloc[1,2]=f"({round(gate.iloc[8,2],4)},{round(gate.iloc[11,2],4)})"
b_df.iloc[1,3]=f"({round(gate.iloc[6,3],4)},{round(gate.iloc[9,3],4)})"
b_df.iloc[1,4]=f"({round(gate.iloc[7,3],4)},{round(gate.iloc[10,3],4)})"
b_df.iloc[1,5]=f"({round(gate.iloc[8,3],4)},{round(gate.iloc[11,3],4)})"

#3rd row: se
b_df.iloc[2,0]=f"[{round(gate.iloc[3,2],4)}]"
b_df.iloc[2,1]=f"[{round(gate.iloc[4,2],4)}]"
b_df.iloc[2,2]=f"[{round(gate.iloc[5,2],4)}]"
b_df.iloc[2,3]=f"[{round(gate.iloc[3,3],4)}]"
b_df.iloc[2,4]=f"[{round(gate.iloc[4,3],4)}]"
b_df.iloc[2,5]=f"[{round(gate.iloc[5,3],4)}]"

b_df.rename(columns={0:f"{gate.columns[2]} 20% Most", 1:f"{gate.columns[2]} 20% Least", 2:f"{gate.columns[2]} Difference", 3:f"{gate.columns[3]} 20% Most", 4:f"{gate.columns[3]} 20% Least", 5:f"{gate.columns[3]} Difference"},inplace=True)

print(b_df)
b_df.to_csv("/content/drive/My Drive/ML/new2/Stage2_gate_satisfaction1.csv", index=False, header=True)



  Booster 20% Most  ... Random Forest Difference
0           0.0579  ...                   0.2046
1  (-0.0083,0.124)  ...          (0.1155,0.3012)
2            [0.0]  ...                    [0.0]

[3 rows x 6 columns]


In [114]:
#GATES p value substitution
if b_df.columns[0]== 'Elastic Net 20% Most':
  b_df.iloc[2,0]='[{:.4f}]'.format(affected_pvalue1[0][0])
  b_df.iloc[2,1]='[{:.4f}]'.format(affected_pvalue1[0][1])
  b_df.iloc[2,2]='[{:.4f}]'.format(affected_pvalue1[0][2])
elif b_df.columns[0]== 'Random Forest 20% Most':
  b_df.iloc[2,0]='[{:.4f}]'.format(affected_pvalue2[0][0])
  b_df.iloc[2,1]='[{:.4f}]'.format(affected_pvalue2[0][1])
  b_df.iloc[2,2]='[{:.4f}]'.format(affected_pvalue2[0][2])
elif b_df.columns[0]== 'Booster 20% Most':
  b_df.iloc[2,0]='[{:.4f}]'.format(affected_pvalue3[0][0])
  b_df.iloc[2,1]='[{:.4f}]'.format(affected_pvalue3[0][1])
  b_df.iloc[2,2]='[{:.4f}]'.format(affected_pvalue3[0][2])
else: 
  b_df.iloc[2,0]='[{:.4f}]'.format(affected_pvalue4[0][0])
  b_df.iloc[2,1]='[{:.4f}]'.format(affected_pvalue4[0][1])
  b_df.iloc[2,2]='[{:.4f}]'.format(affected_pvalue4[0][2])


if b_df.columns[3]== 'Elastic Net 20% Most':
  b_df.iloc[2,3]='[{:.4f}]'.format(affected_pvalue1[0][0])
  b_df.iloc[2,4]='[{:.4f}]'.format(affected_pvalue1[0][1])
  b_df.iloc[2,5]='[{:.4f}]'.format(affected_pvalue1[0][2])
elif b_df.columns[3]== 'Random Forest 20% Most':
  b_df.iloc[2,3]='[{:.4f}]'.format(affected_pvalue2[0][0])
  b_df.iloc[2,4]='[{:.4f}]'.format(affected_pvalue2[0][1])
  b_df.iloc[2,5]='[{:.4f}]'.format(affected_pvalue2[0][2])
elif b_df.columns[3]== 'Booster 20% Most':
  b_df.iloc[2,3]='[{:.4f}]'.format(affected_pvalue3[0][0])
  b_df.iloc[2,4]='[{:.4f}]'.format(affected_pvalue3[0][1])
  b_df.iloc[2,5]='[{:.4f}]'.format(affected_pvalue3[0][2])
else: 
  b_df.iloc[2,3]='[{:.4f}]'.format(affected_pvalue4[0][0])
  b_df.iloc[2,4]='[{:.4f}]'.format(affected_pvalue4[0][1])
  b_df.iloc[2,5]='[{:.4f}]'.format(affected_pvalue4[0][2])



print(b_df, affected_pvalue1, affected_pvalue2, affected_pvalue3, affected_pvalue4)

b_df.to_csv("/content/drive/My Drive/ML/new2/Stage2_gate_satisfaction1.csv", index=False, header=True)


  Booster 20% Most  ... Random Forest Difference
0           0.0579  ...                   0.2046
1  (-0.0083,0.124)  ...          (0.1155,0.3012)
2         [0.1928]  ...                 [0.0000]

[3 rows x 6 columns] [[0.45116978 0.00100369 0.17239502]] [[1.00000000e+00 3.20186540e-08 1.14305318e-05]] [[1.92762476e-01 3.40850298e-11 5.20053013e-09]] [[1.00000000e+00 1.15084645e-06 1.81083188e-03]]
